In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import glob

# Load dataset (same structure as before)
class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        self.preprocessors = preprocessors if preprocessors else []
    
    def load(self, image_paths, verbose=1):
        data, labels = [], []
        class_names = sorted(os.listdir('Helmet_Dataset'))
        
        for class_idx, class_name in enumerate(class_names):
            class_path = os.path.join('Helmet_Dataset', class_name)
            image_paths = glob.glob(os.path.join(class_path, "*.jpg")) + \
                         glob.glob(os.path.join(class_path, "*.png")) + \
                         glob.glob(os.path.join(class_path, "*.jpeg"))
            
            for img_path in image_paths:
                image = cv2.imread(img_path)
                if image is not None:
                    image = cv2.resize(image, (380, 380))  # EfficientNetV2S input size
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    data.append(image)
                    labels.append(class_idx)
        
        return np.array(data), np.array(labels), class_names

print("Loading dataset...")
dl = SimpleDatasetLoader()
(data, labels, class_names) = dl.load('Helmet_Dataset')
print(f"Loaded {len(data)} images: {class_names}")

# Split data
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42, stratify=labels)

# Advanced data augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

# Build EfficientNetV2S (State-of-the-art CNN 2025)
base_model = EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(380, 380, 3))
base_model.trainable = False  # Start with frozen base

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(len(class_names), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7)
]

print("Training EfficientNetV2S model...")
history = model.fit(
    train_datagen.flow(trainX, trainY, batch_size=16),  # Smaller batch for PC
    steps_per_epoch=len(trainX)//16,
    epochs=25,
    validation_data=val_datagen.flow(testX, testY, batch_size=16),
    callbacks=callbacks,
    verbose=1
)

# Unfreeze top layers for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Fine-tune last 20 layers
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower LR for fine-tuning
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Fine-tuning model...")
history_fine = model.fit(
    train_datagen.flow(trainX, trainY, batch_size=16),
    steps_per_epoch=len(trainX)//16,
    epochs=15,
    validation_data=val_datagen.flow(testX, testY, batch_size=16),
    callbacks=callbacks,
    verbose=1
)

# Final evaluation
test_loss, test_acc = model.evaluate(testX, testY)
print(f"\nFinal Test Accuracy: {test_acc:.4f}")

# Save model
model.save('Models/helmet_efficientnetv2.h5')
print("Model saved as 'helmet_efficientnetv2.h5'")


C:\Users\Lenovo\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\Lenovo\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\Lenovo\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Plea

Loading dataset...
Loaded 671 images: ['Helmet', 'Person_no_helmet', 'no_person']
82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 41s 1us/step
Training EfficientNetV2S model...
Epoch 1/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 114s 3s/step - accuracy: 0.8750 - loss: 0.3150 - val_accuracy: 1.0000 - val_loss: 0.1092 - learning_rate: 0.0010
Epoch 2/25
 1/31 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.7143 - loss: 1.2253

C:\Users\Lenovo\AppData\Roaming\Python\Python313\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


31/31 ━━━━━━━━━━━━━━━━━━━━ 22s 706ms/step - accuracy: 0.7143 - loss: 1.2253 - val_accuracy: 1.0000 - val_loss: 0.1111 - learning_rate: 0.0010
Epoch 3/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - accuracy: 0.9815 - loss: 0.0582 - val_accuracy: 0.9940 - val_loss: 0.0545 - learning_rate: 0.0010
Epoch 4/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 24s 714ms/step - accuracy: 1.0000 - loss: 0.0050 - val_accuracy: 1.0000 - val_loss: 0.0493 - learning_rate: 0.0010
Epoch 5/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - accuracy: 0.9897 - loss: 0.0363 - val_accuracy: 0.9940 - val_loss: 0.0352 - learning_rate: 0.0010
Epoch 6/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 27s 814ms/step - accuracy: 1.0000 - loss: 0.0039 - val_accuracy: 0.9940 - val_loss: 0.0337 - learning_rate: 0.0010
Fine-tuning model...
Epoch 1/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 147s 4s/step - accuracy: 0.9795 - loss: 0.0912 - val_accuracy: 1.0000 - val_loss: 0.1088 - learning_rate: 1.0000e-05
Epoch 2/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 23s 696ms/step - accuracy: 0.8750 - lo


Final Test Accuracy: 1.0000
Model saved as 'helmet_efficientnetv2.h5'


In [ ]:
import tensorflow as tf
import cv2
import numpy as np

model = tf.keras.models.load_model('Models/helmet_efficientnetv2.h5')
class_names = ['Helmet', 'no_person', 'Person_no_helmet']

def predict_single_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (380, 380))
    img = np.expand_dims(img, axis=0) / 255.0
    
    predictions = model.predict(img, verbose=0)[0]
    predicted_class = np.argmax(predictions)
    confidence = predictions[predicted_class]
    
    print(f"Predicted: {class_names[predicted_class]}")
    print(f"Confidence: {confidence:.2%}")
    print(f"All probabilities: {dict(zip(class_names, predictions))}")
    return class_names[predicted_class], confidence

# Test
result, conf = predict_single_image('Test_Files/Person_with_helmet_3.jpg')


Predicted: Person_no_helmet
Confidence: 57.22%
All probabilities: {'Helmet': np.float32(0.17703603), 'no_person': np.float32(0.25074992), 'Person_no_helmet': np.float32(0.572214)}
